In [12]:
import scipy.stats as st
np.random.seed(seed=0)

In [17]:
#### Statistical Power
## Benötigte Samplegröße mittels Simulation bestimmen

## Parameter setzen
effect_size     = 0.05
control_mean    = 1
treatment_mean  = control_mean * (1+effect_size)
control_sd      = 0.5

# Zu erziehlende Power
beta = 0.8    

sims = 1000
sample_size = 50

### Idee: 
# Sample size um 10 erhöhen bis benötigte Power erreicht
while 1:
    # Simulieren von Ctrl
    control_time_spent = np.random.normal(loc=control_mean, 
                                          scale=control_sd, 
                                          size=(sample_size, sims))
    # Simulieren von Trmt
    treatment_time_spent = np.random.normal(loc=treatment_mean, 
                                            scale=control_sd, 
                                            size=(sample_size, sims))

    # ##Notiz: 
    # treatment_time_spent = control_time_spent * (1+effect_size) GEHT NICHT!!
    # => nicht ähnlicher Ausgang - braucht nur ca. häfte => siehe nächste Methode

    # t-test
    t, p = st.ttest_ind(treatment_time_spent, control_time_spent)

    # Power = Prozent der Simulationen mit P-Wert kleiner als 0.05
    power = (p < 0.05).sum() / sims
    
    if power >= beta:        # Gewünschte Power erreicht?
        break
    else:
        sample_size += 10    # sonst nächste Simulation mit derzeitiger sample-size + 10
        
print("For {}% power, sample size required = {}".format(beta*100, sample_size))



For 80.0% power, sample size required = 1520


In [20]:
#### Power Berechnung für konkrete Daten 

## benutzen, wenn reale Daten vorhanden auf den man simulieren mächte
# => schliesst die konkreten Charakteristiken der tatsächlichen mit ein
# => kleinerer Gap zwischen Theorie und Praxis

effect_size     = 0.05

# Hier normalerweise die realen Daten rein: 
# => verteilung auf der man den Effekt nachher feststellen soll
control_time_spent_full = np.random.normal(loc=control_mean, 
                                           scale=control_sd, 
                                           size=(sample_size, sims))
# => bei kleiner ausgangsmenge muss bootstrapping mit replace = True gemacht werden
#    ==> liefert sehr ähnliches Ergebnis


### Effekt auf die konkrete population nachstellen!!!
# (anstatt unabhänige mit denselben Parametern zu simulieren)
treatment_time_spent_full = control_time_spent_full * (1+effect_size)

while 1:
    select_index_control = np.random.choice(control_time_spent_full.shape[0], size=sample_size, replace=True)
    control_time_spent = control_time_spent_full[select_index_control]
    select_index_treatment = np.random.choice(treatment_time_spent_full.shape[0], size=sample_size, replace=True)
    treatment_time_spent = treatment_time_spent_full[select_index_treatment]
    #  => jedes mal neues sample generieren!!!

    t, p = st.ttest_ind(treatment_time_spent, control_time_spent)
    power = (p < 0.05).sum()/sims
    if power >= beta:
        break
    else:
        sample_size += 10
        
print("For 80% power, sample size required = {}".format(sample_size))

For 80% power, sample size required = 1640
